_this notebook is meant to execute **`R`** in a `conda` environment_

# Introduction


We end our journey of *higher order* **_Trend Surface analysis with R_** with this Notebook. Here we cover a lot of ground:

<ol type = "1">
    <li>Generalized Least Squares</li>
    
    a) Variogram Modeling and
    b) Ordinary Kriging

  <li>GLS-Regression Kriging</li>
    <li>Universal Kriging</li>
</ol>


<div class="alert alert-block alert-danger"><b>REQUIRED: </b></div>

**You are required to insert your outputs and any comment into this document.**

The document you submit should therefore contain the existing text in addition to:

> *   Plots and other outputs from executing the code chunks  
> *   Discussion of your plots and other outputs as well as conclusions reached.

This should also include any hypotheses and assumptions made as well as factors that may affect your conclusions.

## 1. Generalized Least Squares

At the close of the previous Notebook we talked about **_spatial autocorrelation_**. Patterns and particularly residuals (errors) have a spatial structure / are connected.

What we mean by this is; that even though an estimate _(prediction)_ is unbiased the **structure of the underlying data can be**. A general error term applied to our entire study area might not be ideal.

To overcome the challenge; we use Generalized Least Squares and introduce a **covariance structure between residuals**. Our `linear model`, from [02_extraTrendSurface](https://github.com/AdrianKriger/APG3040C-APG4013C/blob/main/cptFlatsAquifer/02_extraTrendSurface.ipynb) then becomes:

$$
    𝜷_{gls} = (X^TC^{-1}X)^{-1}X^TC^{-1} . y
$$

We see our new term includes $C$; the covariance matrix of the spatially correlated residuals.

The obvious question is. _'OK; so what are the correlated residuals?'_
And the answer is: _'We dont know'_.

What we do is fit the covariance structure (based on imprecise observations) at the same time we calculate the trend surface coefficients.

_please install necessary packages through a `prompt` interface; as per the [requirements.txt](https://github.com/AdrianKriger/APG3040C-APG4013C/blob/main/cptFlatsAquifer/requirements.txt) Not here_.

In [ ]:
options(prompt="> ", continue="+ ", digits=3, width=70, repr.plot.width=7, repr.plot.height=7)
rm(list=ls())

# Install necessary packages: You only need to run this part once
##- install.packages(c("sf", "gstat", "ggplot2", "gridExtra","units", "terra","mgcv","fields","nlme"))

#library(repr)
library(sf)       # 'simple features' representations of spatial objects
library(gstat)    # geostatistics
library(ggplot2)  #
library(gridExtra)
library(units) # units of measure
library(terra) # gridded data structures ("rasters")
library(mgcv)
library(fields)
library(nlme)

In [ ]:
options(show.signif.stars=FALSE)

We've already covered creating a 500m grid and 2nd-order prediction and interpolation with previous Notebooks and move through the introduction quickly.

In [ ]:
file = 'cptFlatsAquifer_watertable4.txt'

In [ ]:
#-- import
cfaq <- read.csv(file, header = 1, sep = ',', dec = '.') #sep = '\t',
#- set crs
cfaq.sf <- st_as_sf(cfaq, coords=c("long", "lat"), crs = 4326) #wgs84
#- transform to local crs
cfaq.sf <- st_transform(cfaq.sf, crs = 32734) #utm 34s
cfaq$X <- st_coordinates(cfaq.sf)[, "X"]
cfaq$Y <- st_coordinates(cfaq.sf)[, "Y"]

model.ts2 <- lm(waterLevel ~ X + Y + I(X^2) + I(Y^2) + I(X*Y), data=drop_units(cfaq))

#--  create grid
(n.col <- length(seq.e <- seq(min.x <- floor(min(cfaq$X)/1000)*1000,
                              max.x <- ceiling(max(cfaq$X)/1000)*1000, by=1000)))
(n.row <- length(seq.n <- seq(min.y <- floor(min(cfaq$Y)/1000)*1000,
                              max.y <- ceiling(max(cfaq$Y)/1000)*1000, by=1000)))

#we want a XXXXm grid
grid <- rast(nrows = n.row, ncols = n.col,
             xmin=min.x, xmax=max.x,
             ymin=min.y, ymax=max.y, crs = st_crs(cfaq.sf)$proj4string,
             resolution = 500, names="waterLevel")

values(grid) <- NA_real_

grid.df <- as.data.frame(grid, xy = TRUE, na.rm = FALSE)
names(grid.df)[1:2] <- c("X", "Y") # match the names of the point dataset
summary(grid.df)

#--
pred.ts2 <- predict.lm(model.ts2,
                       newdata = grid.df,
                       interval = "prediction", level = 0.95)

#-- add the three prediction fields (fit, lower, upper) to the data
grid.df[, 3:5] <- pred.ts2
names(grid.df)[3:5] <- c("ts2.fit", "ts2.lwr", "ts2.upr")

#-- gam
model.gam <- gam(waterLevel ~ s(X, Y, k = 29), data=drop_units(cfaq))
#-- predict aquifer elevation, standard error of prediction using the fitted GAM
tmp <- predict.gam(object=model.gam,
                   newdata=grid.df,
                   se.fit=TRUE)

grid.df$pred.gam <- as.numeric(tmp$fit)
grid.df$pred.gam.se <- as.numeric(tmp$se.fit)

grid.gam <- grid
values(grid.gam) <- grid.df$pred.gam



In [ ]:
#-- look
head(cfaq ,3)

name,long,lat,waterLevel,elevation,depth,X,Y
EM_01,18.6,-34.1,14.4,20.4,63,274222,6227351
EM_27,18.6,-34.1,16.2,20.0,48,276191,6227571
EM_29,18.6,-34.1,18.8,25.5,39,279770,6229369


To compute the trend and covariance at the same time we use the `gls` function in the the `nlme` package.

In [ ]:
#- gls with coefficient
model.ts2.gls <- gls(
  model = waterLevel ~ Y + X + I(Y^2) + I(X^2) + I(X * Y),
  data = drop_units(cfaq),
  method = "ML",
  correlation = corExp(form = ~X + Y,
                       nugget = FALSE,
                       value = 10000) # initial value of the range parameter
)
#model.ts2.gls
class(model.ts2.gls)

In [ ]:
summary(model.ts2.gls)

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

- What is the range of spatial correlation of the exponential model, as estimated by gls?

> [double click in this cell and type your answer here]

**Look at how the GLS and OLS coefficients differ.**

We work with the **Mixed GAM Computation Vehicle** `mgcv` library default smoothing setting and explore **thin plate splines**, as an alternate smoothing function later.

In [ ]:
##-- absolute values
#- generic coef method extracts coefficients from model objects
coef(model.ts2.gls) - coef(model.ts2)

In [ ]:
#- percentage
round(100*(coef(model.ts2.gls) - coef(model.ts2))/coef(model.ts2),1)

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

- Why are the GLS coefficients different than the OLS coefficients?

> [double click in this cell and type your answer here]

**Lets go deeper.**

Display the $90\%$ confidence intervals of the GLS model, calcualte the residuals of this surface and compare those to the OLS.

In [ ]:
#- generic intervals method has a specific method for a fitted GLS model
intervals(model.ts2.gls, level=0.90)

In [ ]:
summary(residuals(model.ts2))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -26.21   -3.82   -1.55    0.00    2.41   29.56 

In [ ]:
summary(res.ts2)

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

- What are the main differences between these sets of residuals? Which surface, in this case, most closely fits the points? Why?

> [double click in this cell and type your answer here]

In [ ]:
#- predict.gls function (of the nlme package) specific method for a fitted GLS model
pred.ts2.gls <- predict(model.ts2.gls, newdata=grid.df)
summary(pred.ts2.gls)

In [ ]:
#- SpatRast of predictions
grid.gls <- grid
values(grid.gls) <- pred.ts2.gls
summary(values(grid.gls))

#- plot
plot(grid.gls, col = rainbow(100), main = "GLS 2nd order predicted surface")
points(st_coordinates(cfaq.sf)[,2] ~ st_coordinates(cfaq.sf)[,1],
       #pch=16,
       #col = ifelse((res.ts2.gls < 0), "red", "green"),
       #cex=2*abs(res.ts2.gls)/max(abs(res.ts2.gls))
       col= ifelse((res.ts2.gls < 0), "red", "green"),
       cex=2*abs(res.ts2.gls)/max(abs(res.ts2.gls)),
       pch=21,
       bg = adjustcolor("black", alpha.f = 0.1),
       #col = "black",
       #cex = 0.9,
       lwd = 0.5)

**Notice how closely this prediction resembles the OLS 2nd-order trend surface!**

Lets see how different?

In [ ]:
#- difference between the OLS and GLS trendsurfaces
grid.gls.ols.diff <- (grid.gls - grid)
plot(grid.gls.ols.diff, col = topo.colors(64),
     main = "GLS - OLS 2nd order trend surfaces, m")

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

-  Where are the largest differences between the OLS and GLS trend surfaces? Explain why.

> [double click in this cell and type your answer here]

Lets look at the residuals

In [ ]:
#- residuals from the GLS trend surface and as a postplot
summary(res.ts2.gls)

In [ ]:
#- plot
plot(cfaq$Y ~ cfaq$X, cex=3*abs(res.ts2.gls)/max(abs(res.ts2.gls)),
     col=ifelse(res.ts2.gls > 0, "green", "red"),
     xlab="X", ylab="Y",
     main="Residuals from 2nd-order trend, GLS fit",
     sub="Positive: green; negative: red", asp=1)
##--  ??local spatial correlation?? --##

**Look at how correlated the residuals are. Nearby data points have similar values**

We now account for the spatial correlation of the residuals

In [ ]:
#-  empirical variogram model residuals from the GLS trend surface model

#- extract the residuals into the point observations object
cfaq.sf$res.ts2.gls <- residuals(model.ts2.gls)
vr.gls <- variogram(res.ts2.gls ~ 1, loc=cfaq.sf, cutoff = 50000)
#- plot empirical variogram
plot(vr.gls, plot.numbers=T,
     main="Residuals from second-order GLS trend", cex=1,
     xlab = "separation [m]", ylab = "semivariance [m^2]")

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

- What are the approximate variogram parameters?

> [double click in this cell and type your answer here]

## 1. a. Variogram Modeling

In order to solve any Kriging formula we need to compute the semivariance at _any_ seperation distance. We want to know the values that will remove the _'overlarge influence'_ nearby datapoints have on a prediction.

In order to do so we fit a variogram function to the empirical variogram. The function represent the structure of spatial autocorrelation. There are many variogram functions: _'Exponential, Spherical, Gauss, Matern'_.  

We will not go into these here but remember to `effective-range / 3` when chooing an _'Exponential'_ model.

In [ ]:
#-  specify variogram model and parameters
vr.gls.m <- vgm(psill=200, model="Mat", range=15000)#, nugget=25)
#vr.gls.m <- vgm(psill=40, model="Exp", range=22000/3, nugget=0)
(vr.gls.m.f <- fit.variogram(vr.gls, vr.gls.m))

In [ ]:
#- plot
plot(vr.gls, model=vr.gls.m.f, plot.numbers=T,
     xlab = "separation [m]", ylab = "semivariance [m^2]")

In [ ]:
print(vr.gls.m.f)

In [ ]:
intervals(model.ts2.gls)$corStruct[2]

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

- Does the range parameter of this fitted model agree with the estimate from the GLS fit?

> [double click in this cell and type your answer here]

## 1. b. The Ordinary Kriging system

Kriging is a form of linear prediction of a value at an unknown point as a weighted sum of values at known points.

This makes sense. We know the structure (spatial correlation) of the residuals, their values and their location. And we use this knowledge to refine our GLS prediction.  

But what really makes Kriging special _(and computationally expensive)_ is the system ensures that each prediction has the least possible prediction variance. The _**uncertainty**_ of the predictions are minimized.   

In [ ]:
#-- ok
grid.sf <- st_as_sf(grid.df, coords = c("X", "Y"))
st_crs(grid.sf) <- st_crs(grid)
kr <- krige(res.ts2.gls ~ 1,
            loc = cfaq.sf,
            newdata = grid.sf,
            model=vr.gls.m.f)

In [ ]:
#- summary
summary(kr)

In [ ]:
class(kr)

In [ ]:
#- ok residuals
plot(kr["var1.pred"], pch=15, nbreaks=24, main="Residuals from GLS trend, m")

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

-  Which areas were most changed by interpolating the residuals? Why?

> [double click in this cell and type your answer here]

In [ ]:
#- prediction standard deviations and plot
kr$var1.sd <- sqrt(kr$var1.var)
summary(kr)

In [ ]:
#-
plot(kr["var1.sd"], pch=15, nbreaks=24, pal = heat.colors,
     main="Standard errors of residuals from GLS trend, m")

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

- Which areas have the most and least uncertainty? Why?

> [double click in this cell and type your answer here]

# 2. GLS-Regression Kriging

We now have the ingredients to predict a very good trend surface. The global function and the local deviations from it.

$$
    Z(s) = Z*(s) + ϵ(s) + ϵ'(s)
$$

We modeled a trend surface, $Z*(s) + ϵ(s)$, with an OLS polynomial and later derived a model for the trend's spatial structure, $ϵ(s) + ϵ'(s)$, by Ordinary Kriging. These were modeled together with GLS.

**We now add the OK prediction of the GLS residuals to the GLS surface. This is called Generalized Least Squares Trend-Regression Kriging (GLS-RK)**

In [ ]:
#-- gls-regression kriging
grid.df$kr <- kr$var1.pred
grid.df$pred.ts2.gls <- pred.ts2.gls
grid.df$rkgls <- grid.df$pred.ts2.gls + grid.df$kr
summary(grid.df)

 num [1:29, 1:2] 274222 276191 279770 283452 282454 ...


In [ ]:
grid.rkgls <- grid
values(grid.rkgls) <- grid.df$rkgls

In [ ]:
#- plot
plot(grid.rkgls, col = rainbow(100),
     main="GLS-RK prediction, aquifer elevation, m.a.s.l.")

Compare the GLS-RK with the GAM

In [ ]:
summary(grid.rkgls.gam <- (grid.rkgls - grid.gam))

In [ ]:
plot(grid.rkgls.gam, main ="RK-GLS - GAM fits, difference, m",
     col = topo.colors(64))

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

- Where are the largest differences between these two trend surface predictions? Explain why, considering how the two surfaces are computed

> [double click in this cell and type your answer here]

## 3. Universal Kriging

We've come to the last leg of our journey.

Universal Kriging (UK) is an extension of OK. UK does not follow a step-wise process of adding an OK prediction of GLS residuals to a GLS surface (like GLS-RK does).

UK bypasses the GLS and predicts a surface from a 2nd-order OLS. In other words the spatial structure of the residuals (the variogram) is not modeled from the GLS residuals but the 2nd-order model directly.

**We start with a clean slate**

In [ ]:
#- what do we have thusfar?
names(grid.sf)

In [ ]:
str(grid.sf$geometry)

In [ ]:
#--
names(cfaq.sf)


In [ ]:
str(cfaq.sf$geometry)

**With UK we must include the coordinates (the predictor variable) in both the observation and prediction grid**

In [ ]:
#--
str(st_coordinates(grid.sf))

In [ ]:
#- variables to grid
grid.sf$X <- st_coordinates(grid.sf)[ , "X"]
grid.sf$Y <- st_coordinates(grid.sf)[ , "Y"]
names(grid.sf)

Variogram from the OLS

In [ ]:
#- empirical variogram
vr <- variogram(zm ~ E + N + I(E^2) + I(N^2) + I(E*N), locations = aq.sf, cutoff = 50000)
#- plot
plot(vr, plot.numbers = TRUE, main = "Residuals from 2nd-order OLS trend surface", xlab = "separation (m)", ylab = "semivariance (m^2)")

In [ ]:
#- variogram model
#(vr.m.f <- fit.variogram(vr, vgm(35, "Exp", 22000/3, 0)))
vr.m.f <- fit.variogram(vr, vgm(psill=200, model="Mat", range=15000)#, nugget=25)

In [ ]:
#- plot
plot(vr, plot.numbers=TRUE, xlab="separation (km)", ylab="semivariance (m^2)", model=vr.m.f, main="Fitted variogram model, residuals from 2nd-order OLS trend surface")


**we can compre the OLS trend surface fit and previous estimate from the GLS fit**

In [ ]:
#- ols residuals
print(vr.m.f)

In [ ]:
#- gls trend residuals
print(vr.gls.m.f)

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

- How do these fitted variogram parameters compare to those from the GLS trend surface residuals? Why are they different?

> [double click in this cell and type your answer here]

In [ ]:
#- uk
k.uk <- krige(zm ~ E + N + I(E^2) + I(N^2) + I(E*N), locations = aq.sf, newdata = grid1km.sf, model=vr.m.f)

In [ ]:
#- plot uk predictors
plot(k.uk["var1.pred"], pch=15, col = rainbow(100), nbreaks=24, main="UK predictions, m")

In [ ]:
#- uk-prediction std-dev.
k.uk$var1.sd <- sqrt(k.uk$var1.var)
summary(k.uk)

In [ ]:
#- plot std-dev
plot(k.uk["var1.sd"], pch=15, nbreaks=24, pal = heat.colors, main="Standard errors of UK predictions, m")

Compare the UK standard deviations to the GLS-Regression Kriging

In [ ]:
#- summary uk
summary(k.uk$var1.sd)

In [ ]:
#- summary gls-rk
summary(kr$var1.sd)

**Note:** the UK should be larger because it includes the uncertainty of the trend surface.

Lets illustrate the UK and GLS-RK differences as a histogram

In [ ]:
#- diff. uk and gls-rk summary
grid.uk <- grid
values(grid.uk) <- k.uk$var1.pred
summary(grid.diff.uk.rkgls <- (grid.uk - grid.rkgls))

In [ ]:
#- diff. uk and gls-rk hist.
hist(grid.diff.uk.rkgls, main = "UK - GLS-RK prediction differences", freq = FALSE, xlab = "difference, UK - GLS-RK")

In [ ]:
#- plot uk minus gls-rk diff surface
plot(grid1km.diff.uk.rkgls, sub="UK - GLS-RK predictions", main="difference, m", xlab="East", ylab="North", col = topo.colors(64))

<div class="alert alert-block alert-warning"><b>QUESTION: </b> </div>

- How large are differences between the UK and GLS-RK trend surface predictions? Where are the largest differences? Explain why there is a difference.

> [double click in this cell and type your answer here]

<div class="alert alert-block alert-success"> <b>DISCUSSION</b>  </div>

 In these series of exercises we have compared several methods of predicting an attribute over space, from a set of geo-referenced observations.

- In this study area, which of the prediction methods would you recommend, and why?

> [double click in this cell and type your answer here]

- For each method introduced, in what situations
would you prefer it to the other methods?

> [double click in this cell and type your answer here]